In [1]:
# from panda3d.core import Point3
# from panda3d.bullet import BulletWorld, BulletBoxShape, BulletRigidBodyNode
# from direct.showbase.ShowBase import ShowBase

# class MyApp(ShowBase):
#     def __init__(self):
#         ShowBase.__init__(self)
        
#         # 初始化 Bullet 物理世界
#         self.bullet_world = BulletWorld()
#         self.bullet_world.setGravity((0, 0, -9.81))  # 设置重力

#         # 加载模型
#         self.model = self.loader.loadModel("models/box")
#         self.model.setScale(1, 1, 1)
#         self.model.setPos(0, 0, 5)
#         self.model.reparentTo(self.render)

#         # 创建 Bullet 物理形状
#         shape = BulletBoxShape((0.5, 0.5, 0.5))

#         # 创建 Bullet 物理节点
#         self.rigid_body_node = BulletRigidBodyNode('box')
#         self.rigid_body_node.addShape(shape)
#         self.rigid_body_node.setMass(1.0)  # 设置质量
#         self.rigid_body_np = self.render.attachNewNode(self.rigid_body_node)

#         # 将物理节点添加到物理世界中
#         self.bullet_world.attachRigidBody(self.rigid_body_node)

#         # 将模型和物理节点同步
#         self.model.reparentTo(self.rigid_body_np)

#         # 设置摄像机位置
#         self.camera.setPos(0, -10, 5)
#         self.camera.lookAt(self.render)

#     def update(self, task):
#         # 更新 Bullet 物理世界
#         dt = globalClock.getDt()
#         self.bullet_world.doPhysics(dt)
#         return task.cont

# app = MyApp()
# app.taskMgr.add(app.update, 'update')
# app.run()


In [2]:
import filepath

In [3]:
# TODO: a room with walls
# a camera that can fly
# from panda3d.core import PointLight, DirectionalLight
import filepath
from util.app import ContextShowBase
from util.log import Loggable
from direct.task import Task
import numpy as np
import torch
from geom.basic import create_cube_node, create_sphere_node,uv_curve_surface, create_colored_cube_node
from panda3d.core import (
    Geom,
    GeomNode,
    GeomTriangles,
    GeomVertexData,
    GeomVertexFormat,
    GeomVertexWriter,
    GeomEnums,
    NodePath,
    PointLight,
    DirectionalLight,
    CardMaker,
    WindowProperties
)
import ursina
from direct.showbase import DirectObject
from game.game_object import GameObject
from game.controller import PlayerController

# controller
# player controller
# agent controller
from typing import Set, List, Dict,Callable
from datetime import datetime
from abc import ABC
from direct.showbase.ShowBase import ShowBase
from panda3d.core import PNMImage, Texture
from panda3d.core import PTAUchar
from panda3d.core import CardMaker
from panda3d.core import Point2
from panda3d.core import NodePath, Camera, PerspectiveLens

import numpy as np
# import gizeh as gz
import random

In [14]:
class CameraController:
    from panda3d.core import (
        LVector3f,
        LQuaternionf
    )
    def __init__(self,camera:Camera):
        self.camera = camera
        self.move_step = .5
        self.turn_step = 5
    # FIXME: use dict
    
    def call_move_forward(self, task):
        self.move(task, self.move_step)
        
    def call_move_backward(self, task):
        self.move(task, -self.move_step)
        
    def call_move_left(self, task):
        new_pos = self.getPos() - self.getRight(self.ref) * self.move_step
        self.setPos(new_pos)
    
        
    def call_move_right(self, task):
        new_pos = self.getPos() + self.getRight(self.ref) * self.move_step
        self.setPos(new_pos)
        
    def call_move_down(self, task):
        self.move_vertically(task, -self.move_step)
        
    def call_move_up(self, task):
        self.move_vertically(task, self.move_step)
    
    def setRef(self, ref):
        self.log("---set ref---:{},{}".format(ref,type(ref)))
        self.ref = ref
        
    def move(self, task:Task, dist:float):
        # self.log("move {}".format(dist))
        new_pos = self.getPos() + self.getForward(self.ref) * dist
        self.setPos(new_pos)
        
    def getForward(self, ref) -> LVector3f:
        return self.getQuat(ref).getForward()
    
    def getRight(self, ref) -> LVector3f:
        print(self.getQuat(ref).getRight())
        return self.getQuat(ref).getRight()
    
    def move_vertically(self, task:Task, dist:float):
        newZ = self.getZ() + dist
        self.setZ(newZ)
        
    def rotate(self, task: Task, angle:float):
        new_H = self.getH() + angle
        self.setH(new_H)
        
    def setPos(self,*args):
        self.camera.setPos(*args)
        
    def setHpr(self, *args):
        self.camera.setHpr(*args)
        
    def setH(self, *args):
        self.camera.setH(*args)
        
    def setP(self, *args):
        self.camera.setP(*args)
        
    def setR(self, *args):
        self.camera.setR(*args)
        
    def setZ(self, *args):
        self.camera.setZ(*args)
    
    def getQuat(self, *args):
        return self.camera.getQuat(*args)
    
    def getPos(self, *args):
        return self.camera.getPos(*args)
    
    def getZ(self, *args):
        return self.camera.getZ(*args)
    
    def toggle_default(self, task):
        pass
    
class PlayerCamController(CameraController, PlayerController):
    def __init__(self, camera): # TODO: input keymap dict
        CameraController.__init__(self, camera) # TODO
        PlayerController.__init__(self)
        # from panda3d.core import KeyboardButton
        self.key_maps = {
            'w': self.call_move_forward,
            's': self.call_move_backward,
            'a': self.call_move_left,
            'd': self.call_move_right,
            'lshift': self.call_move_up,
            'space':self.call_move_down
        }


In [15]:
from util.texture import (
    create_grey_checkerboard, 
    create_color_checkerboard,
    np2texture
)

In [17]:
def make_wall_texture(
    w, h,
    square_size:float=1,
    res:int=8,
    color1 = [0,0,0],
    color2 = [255,255,255],
    name = None
) -> Texture:
    w_img = int(w*res)
    h_img = int(h*res)
    square_size_img = int(square_size*res)
    checkerboard_arr = create_color_checkerboard(
        size=(h_img,w_img),
        square_size=square_size_img,
        color1=color1, color2=color2
    )[:,:,::-1]
    t = np2texture(
        arr=checkerboard_arr,
        format_=Texture.F_rgb,
        name=name
    )
    return t
    

In [27]:
class PhyscRoom(ContextShowBase):
    def __init__(self, xb:int, yb:int , zb:int):
        super().__init__()
        self.name = "Physics Room"
        
        # for controlledShowBase
        self.is_cursor_in_game:bool = True
        self.cam_controller = PlayerCamController(self.camera)
        self.cam_controller.setRef(self.render) ## FIXME: autoset
        
        resolution = 8 # 8pxs a grid
        tex_top = make_wall_texture(
            yb*2,xb*2,square_size=5,
            res=resolution,
            color1 = [135,206,235] # celestial 
        )
        tex_bot = make_wall_texture(
            yb*2,xb*2,square_size=5,
            res=resolution,
            color1 = [227,189,101], # mustard
            color2 = [34,139,34] # grass
        )
        tex_xz = make_wall_texture(
            zb*2,xb*2,square_size=5,
            res=resolution,
            color1= [255,187,255] # plum
        )
        tex_yz = make_wall_texture(
            zb*2,yb*2,square_size=5,
            res=resolution,
            color1 = [0,173,131], # Pantone,
        )        
        
        # maka a room
        cardxy = CardMaker('wallxy')
        cardxy.setFrame(-xb,xb,-yb,yb)
        # FIXME
        cardxy.setUvRange(Point2(0, 1),  # ll
                        Point2(1, 1),  # lr
                        Point2(1, 0),  # ur
                        Point2(0, 0))  # ul
        wall_xy_top = self.render.attach_new_node(cardxy.generate())
        wall_xy_top.set_pos(0,0,zb)
        wall_xy_top.setP(90)
        wall_xy_top.set_texture(tex_top)
        wall_xy_bot = self.render.attach_new_node(cardxy.generate())
        wall_xy_bot.set_pos(0,0,-zb)
        wall_xy_bot.setP(-90)
        wall_xy_bot.set_texture(tex_bot)
        cardxz = CardMaker('wallxz')
        cardxz.setFrame(-xb,xb,-zb,zb)
        cardxz.setUvRange(Point2(0, 1),  # ll
                        Point2(1, 1),  # lr
                        Point2(1, 0),  # ur
                        Point2(0, 0))  
        wall_xz_front = self.render.attach_new_node(cardxz.generate())
        wall_xz_front.set_pos(0,yb,0)
        wall_xz_front.set_texture(tex_xz)
        wall_xz_back = self.render.attach_new_node(cardxz.generate())
        wall_xz_back.set_pos(0,-yb,0)
        wall_xz_back.setH(-180)
        wall_xz_back.set_texture(tex_xz)
        cardyz = CardMaker('wallyz')
        cardyz.setFrame(-yb,yb,-zb,zb)
        cardyz.setUvRange(Point2(0, 1),  # ll
                        Point2(1, 1),  # lr
                        Point2(1, 0),  # ur
                        Point2(0, 0))  
        wall_yz_left = self.render.attach_new_node(cardyz.generate())
        wall_yz_left.set_pos(-xb,0,0)
        wall_yz_left.setH(90)
        wall_yz_left.set_texture(tex_yz)
        wall_yz_right = self.render.attach_new_node(cardyz.generate())
        wall_yz_right.set_pos(xb,0,0)
        wall_yz_right.setH(-90)
        wall_yz_right.set_texture(tex_yz)
        
        self.default_cam_pos = (0, -10, 1)
        self.camera.setPos(*self.default_cam_pos)
        self.accept('z', self.toggle_camera) 
        # self.camera.setHpr(0,-90,0)
        
        
        sphere_o = create_sphere_node("o", 12, 12)
        self.render.attachNewNode(sphere_o)
        node_path_sphere_o = NodePath(sphere_o)
        node_path_sphere_o.setColor((1,1,1,1))
        node_path_sphere_o.setPos(0,0,0)
        sphere_x = create_sphere_node("x", 12, 12)
        self.render.attachNewNode(sphere_x)
        node_path_sphere_x = NodePath(sphere_x)
        node_path_sphere_x.setColor((1,0,0,1))
        node_path_sphere_x.setPos(1,0,0)
        sphere_y = create_sphere_node("y", 12, 12)
        self.render.attachNewNode(sphere_y)
        node_path_sphere_y = NodePath(sphere_y)
        node_path_sphere_y.setColor((0,1,0,1))
        node_path_sphere_y.setPos(0,1,0)
        sphere_z = create_sphere_node("z", 12, 12)
        self.render.attachNewNode(sphere_z)
        node_path_sphere_z = NodePath(sphere_z)
        node_path_sphere_z.setColor((0,0,1,1))
        node_path_sphere_z.setPos(0,0,1)
        node_path_sphere_o.setScale(0.2)
        node_path_sphere_x.setScale(0.2)
        node_path_sphere_y.setScale(0.2)
        node_path_sphere_z.setScale(0.2)
        
        # create a cube
        # self.player_cube =  ControlledCube("player")
        # self.player_cube.reparentTo(self.render)
        # self.player_cube.setZ(self.player_cube.scale/2)
        # self.player_cube.setRef(self.render)
        # self.controller = PlayerCubeController()
        # self.player_cube.register_controller(self.controller)
        self.buttonThrowers[0].node().setButtonDownEvent('button')
        self.buttonThrowers[0].node().setButtonUpEvent('button-up')
        self.accept("space", lambda: print(self.camera.get_pos()))
        self.accept("escape", self.cursor_out)
        self.accept("b", self.cursor_in) # FIXME
        # print("orient", self.player_cube.nodePath.getQuat(render))
        # orient 1 + 0i + 0j + 0k
        # print("orient", self.player_cube.nodePath.getQuat(render).getForward())
        # orient LVector3f(0, 1, 0)
     
        self.cursor_in()
        # 每帧更新摄像机
        self.taskMgr.add(self.update_camera, "update_camera_task")
        # self.taskMgr.add(self.spin_card, "spin_card_task")
        self.taskMgr.add(self.cam_controller.update, "update_controller")

        # 保存鼠标的初始位置
        self.prev_mouse_x = 0
        self.prev_mouse_y = 0
        
    def cursor_in(self):
        # center the mouse
        self.center_mouse()
        # disable default mouse control 
        self.disable_mouse()
        # hide mouse cursor
        props = WindowProperties()
        props.setCursorHidden(True)
        self.win.requestProperties(props)
        # set state of the mouse, 
        # which controls whether camera updates
        self.is_cursor_in_game = True
        
    def cursor_out(self):
        # enable default mouse control
        self.enable_mouse()
        # show mouse cursor
        props = WindowProperties()
        props.setCursorHidden(False)
        self.win.requestProperties(props)
        # set state of the mouse, 
        # which controls whether camera updates
        self.is_cursor_in_game = False

    def center_mouse(self):
        """将鼠标指针重置到窗口的中心"""
        window_center_x = self.win.getXSize() // 2
        window_center_y = self.win.getYSize() // 2
        self.win.movePointer(0, window_center_x, window_center_y)
        self.prev_mouse_x = window_center_x
        self.prev_mouse_y = window_center_y
        
    def toggle_camera(self):
        self.camera.setPos(*self.default_cam_pos)
        self.camera.setHpr(0,0,0)

    def update_camera(self, task):
        """每帧更新摄像机的方向，使其跟随鼠标的移动"""
        if self.mouseWatcherNode.hasMouse() and self.is_cursor_in_game:
            # 获取鼠标的位置（归一化的 -1 到 1 范围内）
            mouse_x = self.win.getPointer(0).getX()
            mouse_y = self.win.getPointer(0).getY()

            # 计算鼠标移动的增量
            delta_x = (mouse_x - self.prev_mouse_x)
            delta_y = mouse_y - self.prev_mouse_y

            # 调整摄像机的水平旋转和俯仰角度
            camera_h = self.camera.getH() - delta_x * 0.1
            camera_p = self.camera.getP() - delta_y * 0.1

            # 设置新的摄像机角度
            self.camera.setH(camera_h)
            self.camera.setP(camera_p)

            # 将鼠标指针重置到窗口的中心
            self.center_mouse()

        return task.cont
    
    def spin_card(self, task):
        time_ = task.time
        angleDegrees = time_ * 60.0

        # self.screen.setH(angleDegrees) # around Z
        # self.screen.setP(angleDegrees)  # around X
        return Task.cont
    
        

#     def spin_camera_task(self, task):
#         # time_ = task.time
#         time_=60
#         angleDegrees = time_ * 60.0
#         angleRadians = angleDegrees * (np.pi / 180.0)
#         self.camera.setPos(20 * np.sin(angleRadians), -20 * np.cos(angleRadians), 3)
#         self.camera.setHpr(angleDegrees, 0, 0)
#         # self.camera.setPos(1,1,3)
#         # self.camera.setHpr(90,0,0)

#         return Task.cont

    def toggle_fullscreen(self):
        props = WindowProperties()
        props.setFullscreen(not self.win.isFullscreen())
        self.win.requestProperties(props)
import builtins
import traceback
try:
    with PhyscRoom(25,25,25) as app:
        app.run()
except Exception as e:
    print(e)
    print(traceback.format_exc())
finally:
    if hasattr(builtins, 'base'):
        builtins.base.destroy()              

Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager


---set ref---:render,<class 'panda3d.core.NodePath'>
---Physics Room run(), at 2024-08-26 00:56:01.220293---
---Physics Room destroy at 2024-08-26 00:56:05.061810---
---Physics Room destroyed at 2024-08-26 00:56:05.062263, exit---


SystemExit: 

In [ ]:
0.87*255cursor_in_game
se